In [1]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import numpy as np
import pandas , xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

/home/pragya/.local/lib/python2.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
Using TensorFlow backend.
/home/pragya/.local/lib/python2.7/site-packages/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)


In [2]:
# csv_file=pd.read_csv('data/corpus_csv.csv')
# csv_file=csv_file.drop(['Unnamed: 0',  'Unnamed: 0.1','title','categories','discipline','videoid','vtt_filename','description','TFIDF_top_words'],axis=1)
# csv_file.describe

In [3]:
# fi=open('data/input.txt','w+')
# fo=open('data/output.txt','w+')
# #empty=[" ",'\n']
# for i in range(len(csv_file)):
#     mystring=csv_file['vtt_Content'][i]
#     mystring = mystring.replace('\n', ' ').replace('\r', '')
#     fi.write(mystring+'\n')
#     #print(str(i)+' written '+csv_file['vtt_Content'][i]+' \n \n \n')
#     fo.write(csv_file['coursename'][i]+'\n')
    

In [4]:
# load the dataset
data = open('data/corpus').read()
labels, texts = [], []
for i, line in enumerate(data.split("\n")):
    content = line.split()
    labels.append(content[0])
    texts.append(content[1:])

# create a dataframe using texts and lables
trainDF = pandas.DataFrame()
trainDF['text'] = texts
trainDF['label'] = labels

In [5]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [6]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
#type(trainDF['text'])
#for eachline in trainDF['text']:
#    for eachword in eachline:
#        eachword.lower()
trainDF['text']=[' '.join(each_text) for each_text in trainDF['text'].values]
#trainDF['text']
count_vect.fit(trainDF['text'])
#type(train_x)
# transform the training and validation data using count vectorizer object
train_x=[' '.join(each_text) for each_text in train_x.values]
valid_x=[' '.join(each_text) for each_text in valid_x.values]
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [7]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(trainDF['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

In [8]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('data/wiki-news-300d-1M.vec')):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(trainDF['text'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [9]:
#trainDF['char_count'] = trainDF['text'].apply(len)
#trainDF['word_count'] = trainDF['text'].apply(lambda x: len(x.split()))
#trainDF['word_density'] = trainDF['char_count'] / (trainDF['word_count']+1)
trainDF['punctuation_count'] = trainDF['text'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
#trainDF['title_word_count'] = trainDF['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
#trainDF['upper_case_word_count'] = trainDF['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [10]:
pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

# function to check and get the part of speech tag count of a words in a given sentence
def check_pos_tag(x, flag):
    cnt = 0
    try:
        wiki = textblob.TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_family[flag]:
                cnt += 1
    except:
        pass
    return cnt

trainDF['noun_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'noun'))
trainDF['verb_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'verb'))
trainDF['adj_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'adj'))
trainDF['adv_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'adv'))
trainDF['pron_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'pron'))


#topic modelling

In [11]:
# train a LDA Model
lda_model = decomposition.LatentDirichletAllocation(n_components=20, learning_method='online', max_iter=20)
X_topics = lda_model.fit_transform(xtrain_count)
topic_word = lda_model.components_ 
vocab = count_vect.get_feature_names()

# view the topic models
n_top_words = 10
topic_summaries = []
for i, topic_dist in enumerate(topic_word):
    topic_words = numpy.array(vocab)[numpy.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))

# model building

In [12]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid,num_epoch, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label,epochs=num_epoch)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [13]:
def train_model_non(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

naive bayes

In [14]:
# Naive Bayes on Count Vectors
accuracy = train_model_non(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print "NB, Count Vectors: ", accuracy

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model_non(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print "NB, WordLevel TF-IDF: ", accuracy

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model_non(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print "NB, N-Gram Vectors: ", accuracy

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model_non(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print "NB, CharLevel Vectors: ", accuracy

NB, Count Vectors:  0.8444
NB, WordLevel TF-IDF:  0.8476
NB, N-Gram Vectors:  0.8328
NB, CharLevel Vectors:  0.8196


#Linear classifier

In [15]:
# Linear Classifier on Count Vectors
accuracy = train_model_non(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
print "LR, Count Vectors: ", accuracy

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model_non(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print "LR, WordLevel TF-IDF: ", accuracy

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model_non(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print "LR, N-Gram Vectors: ", accuracy

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model_non(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print "LR, CharLevel Vectors: ", accuracy

LR, Count Vectors:  0.8656
LR, WordLevel TF-IDF:  0.872
LR, N-Gram Vectors:  0.8308
LR, CharLevel Vectors:  0.8484


#svm

In [16]:
# SVM on Ngram Level TF IDF Vectors
accuracy = train_model_non(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print "SVM, N-Gram Vectors: ", accuracy

SVM, N-Gram Vectors:  0.5192


#bagging model

In [18]:
# RF on Count Vectors
accuracy = train_model_non(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print "RF, Count Vectors: ", accuracy

# RF on Word Level TF IDF Vectors
accuracy = train_model_non(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print "RF, WordLevel TF-IDF: ", accuracy

RF, Count Vectors:  0.7292
RF, WordLevel TF-IDF:  0.7444


# Shallow Neural Net

In [17]:
def create_model_architecture(input_size):
    # create input layer 
    input_layer = layers.Input((input_size, ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(100, activation="relu")(input_layer)
    
    # create output layer
    output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    return classifier 

classifier = create_model_architecture(xtrain_tfidf_ngram.shape[1])
accuracy = train_model(classifier, xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram,5, is_neural_net=True)
print "NN, Ngram Level TF IDF Vectors",  accuracy

Epoch 1/5
7500/7500 [==============================] - 3s 454us/step - loss: 0.5241
Epoch 2/5
7500/7500 [==============================] - 2s 248us/step - loss: 0.2782
Epoch 3/5
7500/7500 [==============================] - 2s 247us/step - loss: 0.1925
Epoch 4/5
7500/7500 [==============================] - 2s 247us/step - loss: 0.1401
Epoch 5/5
7500/7500 [==============================] - 2s 255us/step - loss: 0.1025
NN, Ngram Level TF IDF Vectors 0.5192


# Deep Neural Net

In [18]:
def create_cnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_cnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x,5, is_neural_net=True)
print "CNN, Word Embeddings",  accuracy

Epoch 1/5
7500/7500 [==============================] - 9s 1ms/step - loss: 0.5686
Epoch 2/5
7500/7500 [==============================] - 8s 1ms/step - loss: 0.3914
Epoch 3/5
7500/7500 [==============================] - 8s 1ms/step - loss: 0.3321
Epoch 4/5
7500/7500 [==============================] - 8s 1ms/step - loss: 0.2860
Epoch 5/5
7500/7500 [==============================] - 8s 1ms/step - loss: 0.2556
CNN, Word Embeddings 0.5192


# RNN-LSTM

In [19]:
def create_rnn_lstm():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.LSTM(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rnn_lstm()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x,10, is_neural_net=True)
print "RNN-LSTM, Word Embeddings",  accuracy

Epoch 1/10
7500/7500 [==============================] - 20s 3ms/step - loss: 0.5846
Epoch 2/10
7500/7500 [==============================] - 16s 2ms/step - loss: 0.4794
Epoch 3/10
7500/7500 [==============================] - 19s 3ms/step - loss: 0.4558
Epoch 4/10
7500/7500 [==============================] - 17s 2ms/step - loss: 0.4349
Epoch 5/10
7500/7500 [==============================] - 18s 2ms/step - loss: 0.4158
Epoch 6/10
7500/7500 [==============================] - 21s 3ms/step - loss: 0.3925
Epoch 7/10
7500/7500 [==============================] - 20s 3ms/step - loss: 0.3637
Epoch 8/10
7500/7500 [==============================] - 16s 2ms/step - loss: 0.3519
Epoch 9/10
7500/7500 [==============================] - 16s 2ms/step - loss: 0.3415
Epoch 10/10
7500/7500 [==============================] - 16s 2ms/step - loss: 0.3212
RNN-LSTM, Word Embeddings 0.5192


# RNN-LSTM

In [39]:
def create_rcnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
    
    # Add the recurrent layer
    rnn_layer = layers.Bidirectional(layers.GRU(50, return_sequences=True))(embedding_layer)
    
    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rcnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x,10, is_neural_net=True)
print "CNN, Word Embeddings",  accuracy

Epoch 1/10
7500/7500 [==============================] - 8s 1ms/step - loss: 0.5810
Epoch 2/10
7500/7500 [==============================] - 7s 976us/step - loss: 0.3907
Epoch 3/10
7500/7500 [==============================] - 8s 1ms/step - loss: 0.3306
Epoch 4/10
7500/7500 [==============================] - 7s 984us/step - loss: 0.2772
Epoch 5/10
7500/7500 [==============================] - 7s 989us/step - loss: 0.2361
Epoch 6/10
7500/7500 [==============================] - 8s 1ms/step - loss: 0.1988
Epoch 7/10
7500/7500 [==============================] - 7s 986us/step - loss: 0.1631
Epoch 8/10
7500/7500 [==============================] - 7s 995us/step - loss: 0.1399
Epoch 9/10
7500/7500 [==============================] - 8s 1ms/step - loss: 0.1070
Epoch 10/10
7500/7500 [==============================] - 7s 1000us/step - loss: 0.0940
CNN, Word Embeddings 0.5088
